# Carga de datos

In [1]:
CLUSTER_EVAL=92

In [2]:
!pip install pip install cloudpickle

In [3]:
import pandas as pd
import numpy as np
import gzip, pickle, pickletools
import time
import cloudpickle as cp
import urllib.request

In [4]:
basePath =  'https://www.dropbox.com/s/'

In [5]:
# Dataframe de préstamos
basePath =  'https://www.dropbox.com/s/'
df_prestamos = pd.read_json(basePath + 'q38zr341seq7rkf/joinTablas.json?dl=1')

In [6]:
# Dataframe de datos de material
basePath =  'https://www.dropbox.com/s/'
df_temas = pd.read_json(basePath + 'a2wops7yxdjd2ea/Libros.json?dl=1')

In [7]:
# Dataframe de pesos por los temas de cada cluster6
basePath =  'https://www.dropbox.com/s/'
df_pesos_cluster_temas = pd.read_json(basePath + 'nbz30whibx2r67s/pesos_cluster_temas.json?dl=1')

In [8]:
# Dataframe de pesos por los deweys de cada cluster
basePath =  'https://www.dropbox.com/s/'
df_pesos_cluster_deweys = pd.read_json(basePath + 'f5ixzg087pd8gca/pesos_cluster_deweys.json?dl=1')

In [9]:
# Dataframe de pesos por los deweys de cada cluster
basePath =  'https://www.dropbox.com/s/'
df_pesos_cluster_carreras = pd.read_json(basePath + '4vilvuxp4svg230/pesos_cluster_carreras.json?dl=1')

# Funciones auxiliares

In [10]:
#Para obtener el nombre de un material a partir del título
def itemIdToMaterial(id):
    return df_prestamos.loc[df_prestamos['Llave'] == id]['Titulo'].unique()

In [11]:
def itemIdToDewey(id):
    item = (df_prestamos.loc[df_prestamos['Llave'] == id]['Dewey'].unique())
    if len(item) == 0:
        item = (df_prestamos.loc[df_prestamos['Llave'] == str(id)]['Dewey'].unique())
        if len(item) == 0:
            return -1
        else:
            return item[0]
    else:
        return item[0]

In [12]:
def itemIdToThemes(id):
    tematicas = df_temas.loc[df_temas['Llaves'] == str(id)]['Tema 670'].values
    if len(tematicas) == 0:
        return []
    else:
        return tematicas

In [13]:
def generateClusterInfo(cluster_eval):
    dfP = df_pesos_cluster_temas.copy()
    user_p = dfP.loc[dfP['cluster'] == cluster_eval]
    tematicas =[]
    for llave in user_p['Llaves'].unique():
        tematicas.append(itemIdToThemes((llave)))

    user_p['Tematicas'] = tematicas
    display(user_p)
    deweys = ((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))
    print((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))

    tematicas = []
    for tems in user_p['Tematicas']:
        for tema in tems:
            for temasub in tema:
                if temasub in tematicas:
                    continue
                else:
                    tematicas.append(temasub)

    return user_p, deweys, set(tematicas)

In [14]:
def generateRecsInfopreds(preds):
    recs = preds
    data_recs_asr = []
    for ids in recs:
        data_recs_asr.append({'Titulo':itemIdToMaterial(ids), 
                         #'Rating':ratings[1], 
                          #'Dewey':itemIdToDewey(ids), 
                         'Tematicas':itemIdToThemes(ids),
                          'Llave': ids})

    df_recs_asr = pd.DataFrame(data_recs_asr)
    display(df_recs_asr)
    #deweys = print((df_recs_asr['Dewey'].unique()))
    #print((df_recs_asr['Dewey'].unique()))

    tematicas = []
    for tems in df_recs_asr['Tematicas']:
        for tema in tems:
            for temasub in tema:
                if temasub in tematicas:
                    continue
                else:
                    tematicas.append(temasub)

    #return df_recs_asr, deweys, set(tematicas)
    return df_recs_asr, set(tematicas)

In [15]:
# Compara las tematicas de los ítems recomendados con respecto a los prestamos que ha realizado el cluster
def compare_recs(setRecs, setPrest):
    difference = setRecs - setPrest
    intersection = setRecs & setPrest
    print("Diferencia: ", difference)
    print("Intersección: ", intersection)
    return difference, intersection

In [16]:
df_info_clus, df_clus_deweys, df_clus_tems = generateClusterInfo(CLUSTER_EVAL)

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Llaves,cluster,Dewey,Peso del prestamos,timestamp,Tematicas
1692,255776,92,190,0.250,2018-08-03,[]
1695,255779,92,190,0.250,2019-03-19,"[[filosofía, francés]]"
1701,255783,92,800,0.250,2019-02-13,"[[literatura, moral]]"
1850,256000,92,190,0.125,2019-05-22,"[[filosofía, alemán], [filosofía, alemán]]"
1864,256034,92,130,0.125,2018-11-08,[]
...,...,...,...,...,...,...
179612,885340,92,840,1.000,2019-11-19,"[[poesía, francés]]"
179635,885617,92,840,0.500,2019-01-21,"[[cuento, francés]]"
182378,908913,92,270,0.250,2019-01-21,"[[cristianismo, historia]]"
183156,915537,92,110,0.750,2019-02-06,"[[filosofía, naturaleza]]"


['190' '800' '130' '880' '740' '110' '140' '180' '840' '260' '710' '810'
 '720' '860' '330' '920' '400' '750' '150' '890' '370' '820' '300' '320'
 '780' '170' '520' '590' '20' '500' '120' '830' '0' '460' 'Video' '700'
 '570' '900' '580' '850' '440' '270' '340' '610' '980']


# Modelos

## SAR Temáticas

In [17]:
sar_model_path = basePath + "5brcaex6fdqs3yc/sar_predictor_temas_100k_10011910.pkl?dl=1"

In [18]:
with gzip.open(urllib.request.urlopen(sar_model_path), 'rb') as f:
    p = pickle.Unpickler(f)
    sar_predictor = p.load()

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [19]:
sar_preds = sar_predictor.predict_for_cluster(cluster=CLUSTER_EVAL)

In [20]:
df_info_pred_sar, df_pred_sar_tems = generateRecsInfopreds(sar_preds)

,Titulo,Tematicas,Llave
0,[Entre nosotros ensayos para pensar en otro Em...,"[[filosofía, francés]]",341300
1,[La filosofía de Martín Heidegger Alfonso de W...,"[[filosofía, alemán]]",271907
2,"[Deleuze, Whitehead, Bergson rhizomatic connec...","[[filosofía, francés]]",795173
3,[Deshoras Julio Cortázar],"[[cuento, argentino]]",678233
4,[Rizoma introducción Gilles Deleuze y Pierre F...,"[[filosofía, francés]]",341323
5,[Memorias del subsuelo Fédor Mikhailovitch Dos...,"[[novela, ruso]]",725775
6,[Compendio de psicología William James],[[psicología]],330489
7,"[Obras filosoficas Discurso del metodo, Tratad...","[[filosofía, francés]]",272620
8,[Gilles Deleuze Claire Colebrook],"[[filosofía, francés]]",797150
9,"[El multiculturalismo y ""la política del recon...",[[multiculturalismo]],347754


In [21]:
compare_recs(df_pred_sar_tems, df_clus_tems)

Diferencia:  set()
Intersección:  {'ruso', 'francés', 'filosofía', 'psicología', 'argentino', 'multiculturalismo', 'alemán', 'cuento', 'novela'}


(set(),
 {'alemán',
  'argentino',
  'cuento',
  'filosofía',
  'francés',
  'multiculturalismo',
  'novela',
  'psicología',
  'ruso'})

## SAR Carreras

In [22]:
CARRERA_EVAL = "CARRERA DE INGENIERÍA DE SISTEMAS"

In [23]:
sar_model_carreras_path = basePath + "t5rzulbabu7ca0y/sar_predictor_carreras_100k_10120810.pkl?dl=1"

In [24]:
with gzip.open(urllib.request.urlopen(sar_model_carreras_path), 'rb') as f:
    p = pickle.Unpickler(f)
    sar_predictor_carreras = p.load()

In [25]:
sar_preds_carreras = sar_predictor_carreras.predict_for_cluster(cluster=CARRERA_EVAL)

In [26]:
sar_predictor_carreras.dataset

,userID,itemID,rating,timestamp
0,CARRERA DE DERECHO,199645,0.084746,1.566259e+09
1,CARRERA DE RELACIONES INTERNACIONALES,199645,0.084746,1.554941e+09
2,CARRERA DE CIENCIA DE LA INFORMACIÓN - BIBLIOT...,199810,0.042373,1.525738e+09
3,CARRERA DE CIENCIA DE LA INFORMACIÓN - BIBLIOT...,199825,0.042373,1.583280e+09
4,CARRERA DE ESTUDIOS MUSICALES,200299,0.021186,1.546430e+09
...,...,...,...,...
99995,MAESTRÍA EN Filosofía,796341,0.084746,1.569888e+09
99996,CARRERA DE ECOLOGÍA,796343,0.042373,1.548115e+09
99997,DEPARTAMENTO DE INGENIERÍA CIVIL,796343,0.127119,1.563322e+09
99998,LICENCIATURA EN TEOLOGÍA,796346,0.127119,1.563840e+09


In [27]:
sar_preds_carreras

72057    713692
58739    668826
10569    279206
89746    762003
16830    301931
15236    296025
74685    722029
13622    288369
72141    714070
8235     274655
Name: itemID, dtype: object

In [28]:
df_info_pred_carreras_sar, df_pred_carreras_sar_tems = generateRecsInfopreds(sar_preds_carreras)

,Titulo,Tematicas,Llave
0,[[Shin Bunka Nihongo]],"[[japonés, enseñanza], [japonés, enseñanza]]",713692
1,"[IELTS practice tests James Milton, Huw Bell a...","[[inglés, enseñanza, estudiante, extranjero], ...",668826
2,[Gargantua Francois Rebelais],"[[novela, francés]]",279206
3,[[Dulce de queso] Konami Kanata],"[[japonés, libro, lectura], [japonés, libro, l...",762003
4,[Curso de títulos valores jurisprudencia-doctr...,"[[documento, negociable]]",301931
5,"[Tus zonas erroneas Wayne W. Dyer, Datos sobre...","[[seguridad, psicología]]",296025
6,"[Comedia Dante Alighieri ; traducción, prólogo...","[[poesía, italiano], [poesía, italiano], [poes...",722029
7,[Introducción a la astronomia el sistema solar...,"[[mecánico, celeste]]",288369
8,[Tratado de fisiología médica Arthur Clifton G...,"[[fisiología, humano]]",714070
9,[Punto y línea sobre el plano contribución al...,"[[pintura, investigación]]",274655


In [29]:
compare_recs(df_pred_carreras_sar_tems, df_clus_tems)

Diferencia:  {'negociable', 'extranjero', 'documento', 'seguridad', 'fisiología', 'japonés', 'lectura', 'humano', 'mecánico', 'estudiante', 'celeste', 'enseñanza', 'libro'}
Intersección:  {'inglés', 'francés', 'italiano', 'psicología', 'pintura', 'investigación', 'poesía', 'novela'}


({'celeste',
  'documento',
  'enseñanza',
  'estudiante',
  'extranjero',
  'fisiología',
  'humano',
  'japonés',
  'lectura',
  'libro',
  'mecánico',
  'negociable',
  'seguridad'},
 {'francés',
  'inglés',
  'investigación',
  'italiano',
  'novela',
  'pintura',
  'poesía',
  'psicología'})

In [30]:
%load_ext autoreload
%autoreload 2

# Dash app

In [31]:
!pip install jupyter-dash

In [32]:
from jupyter_dash import JupyterDash

In [33]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  
/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  after removing the cwd from sys.path.


In [34]:
available_list = ['Cluster por temáticas', 'Programas académicos']

In [35]:
available_groups_temas = df_pesos_cluster_temas['cluster'].unique()

In [36]:
available_groups_carreras = df_pesos_cluster_carreras['cluster'].unique()

In [37]:
df_info_pred_sar

,Titulo,Tematicas,Llave
0,[Entre nosotros ensayos para pensar en otro Em...,"[[filosofía, francés]]",341300
1,[La filosofía de Martín Heidegger Alfonso de W...,"[[filosofía, alemán]]",271907
2,"[Deleuze, Whitehead, Bergson rhizomatic connec...","[[filosofía, francés]]",795173
3,[Deshoras Julio Cortázar],"[[cuento, argentino]]",678233
4,[Rizoma introducción Gilles Deleuze y Pierre F...,"[[filosofía, francés]]",341323
5,[Memorias del subsuelo Fédor Mikhailovitch Dos...,"[[novela, ruso]]",725775
6,[Compendio de psicología William James],[[psicología]],330489
7,"[Obras filosoficas Discurso del metodo, Tratad...","[[filosofía, francés]]",272620
8,[Gilles Deleuze Claire Colebrook],"[[filosofía, francés]]",797150
9,"[El multiculturalismo y ""la política del recon...",[[multiculturalismo]],347754


In [38]:
!pip install dash-bootstrap-components

In [42]:
# Build App
import dash_bootstrap_components as dbc
print(dbc.themes.CYBORG)
# app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Recomendaciones para grupos definidos"),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df_info_pred_sar.columns],
        data=df_info_pred_sar.to_dict('records'),
    ),
    html.Label([
        "Lista a la que recomendar",
        dcc.Dropdown(
            id='list-dropdown', clearable=False,
            value = available_list, options=[
                {'label': i, 'value': i}
                for i in available_list
                
            ])
    ]),
    html.Label([
        "Grupo al que recomendar",
        dcc.Dropdown(
            id='group-dropdown', clearable=False,
            value = available_groups_temas, options=[
                {'label': i, 'value': i}
                for i in available_groups_temas
                
            ])
    ]),
])

# Callback para actualizalos valores de grupos
@app.callback(
    Output('group-dropdown', 'options'),
    Input('list-dropdown', 'value')
)
def update_options(value):
    print("Valor de lista", value)
    if(value == 'Cluster por temáticas'):
        return [{'label': i, 'value': i} for i in available_groups_temas]
    else:
        return [{'label': i, 'value': i} for i in available_groups_carreras]
    

# Definecallback to update graph
@app.callback(
    Output('table', 'data'),
    [Input("list-dropdown", "value"),
    Input("group-dropdown", "value"),]
)
def update_figure(lista, grupo):
    if (lista == 'Cluster por temáticas'):
        sar_preds = sar_predictor.predict_for_cluster(cluster=grupo)
        df_info_pred_sar, df_pred_sar_tems = generateRecsInfopreds(sar_preds)
        columns=[{"name": i, "id": i} for i in df_info_pred_sar.columns]
        data=df_info_pred_sar.to_dict('records')
        return data
    else:
        sar_preds = sar_predictor_carreras.predict_for_cluster(cluster=grupo)
        df_info_pred_sar, df_pred_sar_tems = generateRecsInfopreds(sar_preds)
        columns=[{"name": i, "id": i} for i in df_info_pred_sar.columns]
        data=df_info_pred_sar.to_dict('records')
        return data
        


https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/cyborg/bootstrap.min.css


In [43]:
app.run_server(mode='external', port='8050')
%tb

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/


No traceback available to show.


Valor de lista ['Cluster por temáticas', 'Programas académicos']
Valor de lista ['Cluster por temáticas', 'Programas académicos']
Valor de lista Programas académicos


,Titulo,Tematicas,Llave
0,[],"[[música, cámara], [música, cámara], [música, ...",200789
1,[],[],753322
2,[La evolución de la conciencia Stanislav Grof....,"[[evolución, humano]]",344472
3,[Los cronistas Adolfo Milanes],"[[ocana, norte, santander, colombia, biografias]]",335505
4,[Mister cuervo Luísa Morandeira ; ilustracione...,"[[cuento, infantil, español]]",762489
5,[],"[[música, película, cinematográfico, estudio, ...",795782
6,[],"[[violín, estudio, enseñanza], [violín, estudi...",200644
7,[Teaching music in the twenty-first century Lo...,"[[música, enseñanza]]",601828
8,"[Music and Marx ideas, practice, politics edit...","[[música, aspecto, social]]",741544
9,[A practical approach to eighteenth-century co...,"[[contrapunto, historia, siglo, xviii]]",758161


...

,Titulo,Tematicas,Llave
0,[Rehabilitación implanto-asistida bases y fund...,"[[implant, dental]]",765601
1,[Teoría general del proceso aplicable a toda c...,"[[derecho, procesal], [estética, dental]]",541308
2,[Oral and maxillofacial trauma edited by Raymo...,"[[maxilar, malformación], [maxilar, malformaci...",310149
3,"[Pensar bien, sentirse bien Walter Riso ; edic...",[[autoestima]],683800
4,[Principles of oral and maxillofacial surgery ...,"[[cirugía, bucal]]",718382
5,[Odontopediatria : resoluçöes clínicas Faculda...,"[[odontología, pediátrico]]",666210
6,[],"[[visualización, información]]",239196
7,[Reconstructive preprosthetic oral and maxillo...,"[[prótesis, dental]]",347481
8,[Manual de habilidades para auditoría una intr...,"[[auditoría, manual]]",668305
9,[En un origen las palabras eran magia por Stev...,"[[psicoterapia, familiar]]",701219


,Titulo,Tematicas,Llave
0,[Introducción a la historia de las relaciones ...,"[[relación, internacional, historia]]",750030
1,[Tratado sobre la familia Gary Stanley Becker ...,"[[familia, aspecto, socioeconómico]]",536422
2,[Macroeconomía N. Gregory Mankiw ; traducción ...,[[macroeconomía]],751407
3,[Introducción a la integración económica Herna...,"[[integración, económico]]",640548
4,[Maquinado de la madera Servicio Nacional de A...,"[[ebanistería, equipo, accesorio]]",629338
5,"[Neurociencia y conducta Eric R. Kandel, James...",[[neurociencia]],355448
6,[Museo de arte colonial Universidad Nacional d...,"[[museo, arte, colonial, historia, santo, fe, ...",323728
7,[El libro de los oficios de antaño Eduardo Santa],"[[empleos, colombia, historia]]",532529
8,[],"[[película, cinematográfico, unidos]]",793967
9,[Desempeño por competencias evaluación de 360À...,"[[evaluación, personal]]",705252


Valor de lista Cluster por temáticas


""


,Titulo,Tematicas,Llave
0,[Situated literacies reading and writing in co...,"[[analfabetismo, aspecto, social]]",804495
1,[La vejez a debate analisis historico de la si...,[],352406
2,[La enfermedad y sus metáforas y ; El sida y s...,"[[enfermedad, aspecto, social]]",760044
3,[Cuerpos que importan sobre los límites materi...,[[feminismo]],698891
4,"[Nuevos paradigmas, cultura y subjetividad Enc...","[[teoría, conocimiento, congreso]]",348508
5,[El político y el cientifico Max Weber ; Intro...,"[[ciencia, político]]",285059
6,"[Investigación de mercados David A. Aaker, V. ...","[[derecho, romano], [investigación, mercado]]",717850
7,[],"[[música, guitarra, arreglo]]",631344
8,"[Géneros periodísticos Juan Gargurevich Regal,...",[],544762
9,[Historia de la enfermería evolución histórica...,"[[enfermería, historia]]",725608


In [41]:
!ps -fA | grep python

  501 64678 64666   0 Sat11PM ??        34:14.17 /opt/anaconda3/envs/reco_base/bin/python -m ipykernel_launcher -f /Users/danieltibaquira/Library/Jupyter/runtime/kernel-01838f45-f4e7-46da-a370-33c72b33e3c9.json
  501 69948 64666   0  9:57PM ??         0:00.75 /opt/anaconda3/envs/reco_base/bin/python -m ipykernel_launcher -f /Users/danieltibaquira/Library/Jupyter/runtime/kernel-5bb21fe8-2f6b-412f-a592-da63047d4d1c.json
  501 72247 64666   0  7:39AM ??         1:38.88 /opt/anaconda3/envs/rec_base/bin/python -m ipykernel_launcher -f /Users/danieltibaquira/Library/Jupyter/runtime/kernel-03ab8a84-f82e-4cdc-9599-cec6be5d1945.json
  501 64666 32386   0 Sat11PM ttys000    0:51.21 /opt/anaconda3/envs/rec_base/bin/python3.7 /opt/anaconda3/envs/rec_base/bin/jupyter-notebook
  501 72323 72247   0  7:42AM ttys002    0:00.01 /bin/zsh -c ps -fA | grep python
  501 72326 72323   0  7:42AM ttys002    0:00.00 grep python
